In [1]:
# topic 추출

In [83]:
import json
import os
import glob

In [84]:
directory = os.getenv('HOME') + '/aiffel/aiffelthon/'

In [92]:
len(glob.glob(directory + 'reg_5/*.txt'))

4717

In [8]:
topic_txt_file = './topic_txt_file.txt'

In [9]:
# with open(topic_txt_file, 'w') as txt:
#     for i in topics :
#         txt.write(i + ',\n')

In [15]:
with open(topic_txt_file) as f :
    raw = f.read().replace(',','').splitlines()
raw

['날씨',
 '공연',
 '정치',
 '연애/결혼',
 'SNS',
 '스포츠/레저',
 '콤플렉스',
 '드라마',
 '반려동물',
 '아르바이트',
 '먹거리',
 '취미',
 '방송/연예',
 '건강',
 '만화',
 '선물',
 '자동차/오토바이',
 '4차산업혁명',
 '꿈(목표)',
 '가족',
 '좌우명',
 '문학',
 '회사/학교',
 '재테크',
 '전자기기',
 '쇼핑',
 '법률',
 '역사',
 '다이어트',
 '음악',
 '게임',
 '군대',
 '영화',
 '계절',
 '종교',
 '여행지(국내/해외)',
 '경제',
 '스타일',
 '자격증',
 '성격']

In [16]:
final_dir = directory + '/final'

In [17]:
os.listdir(final_dir)

['raw_data_sampling(0223).csv',
 'Train_set_data.csv',
 'Test_set_data_sampling(0223).csv']

In [18]:
import pandas as pd
df = pd.read_csv(final_dir + '/Test_set_data_sampling(0223).csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    500 non-null    object
 1   dial    500 non-null    object
 2   reg     500 non-null    object
 3   eng     500 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB


In [20]:
df.reg.value_counts()

jj    100
cc    100
kw    100
jd    100
gs    100
Name: reg, dtype: int64

In [21]:
from transformers import pipeline

In [22]:
# model_ckpt = 'ddobokki/klue-roberta-base-nli-sts'
# model_ckpt = 'Huffon/klue-roberta-base-nli'
model_ckpt ='snunlp/KR-SBERT-V40K-klueNLI-augSTS'

In [23]:
pipe = pipeline('fill-mask', model =model_ckpt)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at snunlp/KR-SBERT-V40K-klueNLI-augSTS and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
test = df['text'][469]
test

'뭐~ 요리사라든 이렇게 쪼금 어~ 뭐~ 잘하시는 요리 조금 자신있 있으신 요리가 있으신지 조금 궁금합니다.'

In [72]:
all_labels = raw

In [79]:
# ! pip install sentence_transformers
from sentence_transformers import SentenceTransformer,util


model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS', device=0)

def topic_select(x, all_labels) :
    
    embeddings = model.encode([x], convert_to_tensor=True) # 문장 
    tgt_embddings = model.encode(all_labels, convert_to_tensor=True) # topic
    top = torch.argmax(util.pytorch_cos_sim(embeddings[0], tgt_embddings)).item()
    
    return all_labels[top]

In [81]:
df['topic'] = df['text'].apply(lambda x : topic_select(x, all_labels))
df

text  \
0                            그니까 거기 그 사진 찍는아니야 주상절리 마냥   
1                                               집에 있으면   
2                      먹고 취하잖아 그러면 취하면 한이 있잖아 사람이 왜 이게   
3                                           그렇지 그렇게하겠지   
4                나 아는사람이라고 해서 십만 원 더 지원해서 이십만 원 지원해 줬어   
..                                                 ...   
495  뭐~ 두 분 쓰시는 게 없어서 뭐~ 안정적인 건지 살 잘 얘기를 안 하시더라고요 그...   
496                   잘 치거나 이렇게 했을 때 다 그냥 이렇게 손 한 번씩 다   
497                                        아니 잘 안 맞지 넌   
498                   진짜 너무 이거 조금 조금 미안하기도 하고 다른 사람들한테   
499  바퀴벌레가 방역을 해도 기어나와 가지고 이제는 쪼끔 마음이 조금 나아지긴 했는데 처...   

                                                  dial reg  \
0                             그니까 거기 그 사진 찍는아니 주상절리 마냥  jj   
1                                               집에 이시믄  jj   
2                   먹어지믄 취하잖아 겅하*믄이 취하면 한이 있네 사람이 왜 이게  jj   
3                                            그렇지 겅하겠주게  jj   
4                 나 아는사람이라고 해서 십만 원 더 지원해서 이십만 원 지원해 줜  jj   
..                                                 ...  ..   
495  뭐~ 두 분 쓰시는 게 없어서 뭐~ 안정적인 건지 살 잘 얘기를 안 하시더라고요 그...  gs   
496                    잘 치거나 이렇게 했을 때 다 그냥 이케 손 한 번씩 다  gs   
497                                        아니 잘 안 맞지 닌  gs   
498                   진짜 너무 이거 쪼끔 쪼끔 미안하기도 하고 다른 사람들한테  gs   
499  바퀴벌레가 방역을 해도 기어나와 가지고 이제는 쪼끔 마음이 쫌 나아지긴 했는데 처음...  gs   

                                                   eng topic  
0    So youre not taking that picture there are you...    공연  
1                                   If you are at home    가족  
2    You're eating and getting drunk, and if you're...    역사  
3                              Thats right Ill do that    경제  
4    I ot it as I know so I applied for another 100...    선물  
..                                                 ...   ...  
495  Im worried about whether its stable or not bec...  콤플렉스  
496  When you hit well or did this, you're all just...    선물  
497                          No it doesnt fit well you   좌우명  
498                           I'm so sorry about this.    선물  
499  I fot it out even if the cockroaches crawled o...  전자기기  

[500 rows x 5 columns]

In [82]:
df.head()

text  \
0              그니까 거기 그 사진 찍는아니야 주상절리 마냥   
1                                 집에 있으면   
2        먹고 취하잖아 그러면 취하면 한이 있잖아 사람이 왜 이게   
3                             그렇지 그렇게하겠지   
4  나 아는사람이라고 해서 십만 원 더 지원해서 이십만 원 지원해 줬어   

                                   dial reg  \
0              그니까 거기 그 사진 찍는아니 주상절리 마냥  jj   
1                                집에 이시믄  jj   
2    먹어지믄 취하잖아 겅하*믄이 취하면 한이 있네 사람이 왜 이게  jj   
3                             그렇지 겅하겠주게  jj   
4  나 아는사람이라고 해서 십만 원 더 지원해서 이십만 원 지원해 줜  jj   

                                                 eng topic  
0  So youre not taking that picture there are you...    공연  
1                                 If you are at home    가족  
2  You're eating and getting drunk, and if you're...    역사  
3                            Thats right Ill do that    경제  
4  I ot it as I know so I applied for another 100...    선물